# Temiloluwa Philip Ojo
##  codekagei@gmail.com

### Assignment 2 solution 

In [ ]:
# Import the necessary package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler,StandardScaler
# data splitting into train and test set
from sklearn.model_selection import train_test_split
%matplotlib inline

In [ ]:
# read the defects data
android_data = pd.read_csv('./android_traffic.csv')
android_data.head()

In [ ]:
# discover the shape

print("This dataset has {} rows and {} columns".format(android_data.shape[0],android_data.shape[1]))

# discover the datatypes
print(android_data.dtypes)

In [ ]:
types = android_data.dtypes
print("Number categorical featues:", sum(types=='object'))
print("Number non-categorical featues: {}".format(android_data.shape[1] - sum(types == 'object')))

In [ ]:
# description of the data
android_data.describe()

In [ ]:
# find missing values
print(android_data.isna().any().any())

# find count of missing values
print(android_data.isna().sum().sum())

In [ ]:
# Transformation of categorical features to integers

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, drop='first')
cat_feats = ['type']
encoder.fit(android_data[cat_feats])

def ohe_new_features(df, features_name, encoder):
    new_feats = encoder.transform(df[features_name])
    new_cols = pd.DataFrame(new_feats, dtype=int, columns=encoder.get_feature_names(features_name))
    new_df = pd.concat([df, new_cols], axis=1)
    new_df.drop(features_name, axis=1, inplace=True)
    return new_df

android_data = ohe_new_features(android_data,cat_feats,encoder )

In [ ]:
# One hot encoding of categorical data completed.
android_data.head()

In [ ]:
# seperate features from target
X = android_data.drop(['type_malicious'], axis=1)
y = android_data['type_malicious']

In [ ]:
# feature scaling
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15)

### Question 1.3
Before you start building the models, analyze the data (see the number of features,
etc.) and tell us whether you expect kNN or SVM to perform better on this problem.
Justify your answer.

I expect KNN to perform better for this dataset since it contains a small amount 
of features which it could be inferred that there is less dimensionality than a dataset with a 
high number of features and KNN works well with small dataset as it loads all the data into memory
while SVM is best for data containing high dimensionality. 

In addition, there are 7845 training data  and 12 features, this makes KNN more suitable as it 
does better than SVM when the number of training data is greater than the number of features.

### Question 1.4

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print(accuracy_score(y_test, y_pred))

In [ ]:
## hyper parameter tuning

from sklearn.model_selection import GridSearchCV

param_grid = {'n_neighbors':list(range(1, 10)),
              'weights':['distance','uniform'],
              'metric':['euclidean', 'manhattan', 'chebyshev', 'cosine']
              }

grid_search_clf = GridSearchCV(estimator=KNeighborsClassifier(), cv=7, 
                               scoring='accuracy',param_grid=param_grid
                                )

grid_search_clf.fit(x_train, y_train)

means = grid_search_clf.cv_results_['mean_test_score']
stds = grid_search_clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, grid_search_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
    
print()
print("The best parameters set found on development set:")
print()
print(grid_search_clf.best_params_)

y_pred = grid_search_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

## Question 1.5 

Implement SVM for malware detection. You must perform hyper-parameter tuning
for your model on the following hyper-params: {C: np.linspace(1, 2, num=5), kernel:
[’rbf ’, ’sigmoid’], gamma: np.logspace(-7, -3, num=5)}. Print the mean score and its
standard deviation for each of the built models. Print the best hyper-parameters.

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"

# hyper parameter tuning with grid search on SVM
param_grid =  {'C': np.linspace(1, 2, num=5), 
               'kernel': ['rbf', 'sigmoid'], 
               'gamma': np.logspace(-7, -3, num=5)}

grid_search_clf = GridSearchCV(estimator=SVC(),
                               cv=7, scoring='accuracy',
                               param_grid=param_grid
                               )

grid_search_clf.fit(x_train, y_train)

means = grid_search_clf.cv_results_['mean_test_score']
stds = grid_search_clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, grid_search_clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
        % (mean, std * 2, params))
    
print()
print("The best parameters set found on development set:")

print()

print(grid_search_clf.best_params_)

y_pred = grid_search_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

## Question 1.6
Train both kNN and SVM using the best hyper-parameters on the train set, and print
the accuracy on the test set for both models. Do the results support your answer to
1.3?

In [ ]:
# KNN model with the best parameters
knn_model = KNeighborsClassifier(metric='cosine', n_neighbors=9, weights='distance')

knn_model.fit(x_train, y_train)

y_pred = knn_model.predict(x_test)

knn_model_accuracy_score = accuracy_score(y_test, y_pred)

In [ ]:
# SVM model with the best parameters
svc_model = SVC(C=1.0, gamma=0.001, kernel='rbf')

svc_model.fit(x_train, y_train)

y_pred = svc_model.predict(x_test)

svc_model_accuracy_score = accuracy_score(y_test, y_pred)

In [ ]:
# Acccuracy score of both models after application of the best hyper parameters
print("The accuracy score of the KNN model using the best hyperparameters is: ", knn_model_accuracy_score)

print("The accuracy score of the SVM model using the best hyperparameters is: ", svc_model_accuracy_score)

### Discussion

From the above result, the accuracy score of KNN is more than that of SVM. 
This is in line with my assumption of which model will perform better when observing the 
count of the total training data and number of features in question **1.3**

## Question 2

In [ ]:
# package imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD

### 2.1 
Construct a DNN model that has one input layer, one output layer, and 3 hidden
layers. The number of nodes in the input layer would be the same as number of
statements in the coverage data, whereas the number of nodes in each hidden layer
should be simply set as 32. The output layer would have just one node (representing
the execution result r) with sigmoid activation function. The activation function in
the hidden layers should be the relu function.

In [ ]:
# read the data
coverage_data = pd.read_csv('./coverage.csv')
coverage_data.head()

In [ ]:
# discover the shape

print("This dataset has {} rows and {} columns".format(coverage_data.shape[0],coverage_data.shape[1]))

# discover the datatypes
print(coverage_data.dtypes)

In [ ]:
X = coverage_data.drop(['Unnamed: 0','r'], axis=1)
y = coverage_data['r']

In [ ]:
# get the number of statements to use for the nodes in the input layer
input_layer_size = X.shape[1]
batch_size = 10
epochs = 30
print('The input size is ', input_layer_size)
print('The batch size is ', batch_size)
print('The epochs is',epochs)

In [ ]:
def get_fl_model():
    model = Sequential()

    model.add(Dense(units=32, input_dim=input_layer_size, activation='relu'))
    model.add(Dense(units=32,  activation='relu'))
    model.add(Dense(units=32,  activation='relu'))
    model.add(Dense(units=1,use_bias=True, activation='sigmoid'))
    return model

In [ ]:
model = get_fl_model()

In [ ]:
model.summary()

## Question 2.2
Use the coverage data as the training set to fit/train your DNN model to obtain the
complex nonlinear mapping relationship between the coverage data of a test case and
its execution result. Regarding the hyper-parameters: Optimizer=Adam(lr=1e-5),
epochs=30.

In [ ]:
adam_optimizer = Adam(lr=1e-5)

# compiling the keras model
model.compile(loss='binary_crossentropy', optimizer=adam_optimizer, metrics=['accuracy'])

In [ ]:
# split data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38)

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=batch_size, epochs=epochs)

loss, acc = model.evaluate(x_test, y_test)
print("Loss:", loss, ", Accuracy:", acc)

In [ ]:
## Saving the wrights of the model in hdf5
model.save("my_model.pth")
model.save("my_model.hdf5")

### Question 2.3

In this step, you will construct the set of virtual test cases. Once again, to get a better
understanding of this set, refer to [1]. In summary, this set will have a structure as
shown in Figure 2, consisting of the same number of statements as in the coverage
data. Note that here each row would represent a virtual test case which covers only
one statement of the given computer program. Since each test case only covers one
statement, thus the statement would be highly suspicious if the corresponding test
case is failed.

In [ ]:
### First step is to build a virtual test case data

virtual_data = np.zeros((500,500),int)
np.fill_diagonal(virtual_data,1)

print("This virtual dataset has {} rows and {} columns".format(virtual_data.shape[0],virtual_data.shape[1]))


### Question 2.4
Input the coverage data vector of virtual test set to the trained DNN model and get

the output for each test case, which would reflect the probability that its correspond-
ing executable statement has a bug. Finally, rank the statements according to the

predicted probability scores.

In [ ]:
prediction_on_virtual_data = model.predict(virtual_data)

In [ ]:
# round the predictions to 2 decimal places to make it easier to work with the data
prediction_on_virtual_data = [round(x[0],4) for x in prediction_on_virtual_data]

In [ ]:
print(prediction_on_virtual_data)

In [ ]:
# Create an array containing the statement number and the corresponding suspiciousness

state_susp_array = []

for i in range(len(prediction_on_virtual_data)):
#     since indexes start at 0, 1 is added to each index for the statement
    state_susp_array.append([i + 1 , prediction_on_virtual_data[i]])

print(state_susp_array[:20])

In [ ]:
# sort the statement and suspiciousness according to the suspiciousness
# this would make it easy to get the rank of each suspiciousness
state_susp_array = sorted(state_susp_array, key=lambda state_susp_array: state_susp_array[1],reverse=True)

# after sorting based on suspiciousness
print(state_susp_array[:20])

In [ ]:
# get the rank of each statement and append it to the statement and suspiciousness

for i in range(len(state_susp_array)):
    #     since indexes start at 0, 1 is added to each index for the rank
    state_susp_array[i].append(i + 1)

In [ ]:
print(state_susp_array[:20])

In [ ]:
# sort the data based on the statement

state_susp_rank = sorted(state_susp_array, key=lambda state_susp_array: state_susp_array[0])

In [ ]:
print(state_susp_rank[:20])

In [ ]:
# create a data frame containing the statement, suspiciousness and rank
state_susp_rank_df = pd.DataFrame(state_susp_rank, columns=['Statement','Suspiciousness','Rank'])

In [ ]:
state_susp_rank_df.head(20)

In [ ]:
state_susp_rank_df.to_csv('state_susp_rank_df.csv',header=True, index=False)

### Question 2.6
(Bonus task) Tune the following hyper-parameters to achieve the best accuracy on a
validation split (15% of the training set).
- Number of layers.
- Number of neurons in each layer.
- adding dropout layers.
- optimizer and learning rate.
- activation functions.

Each hyper parameter will be fine tuned seperately and used to tune
other hyper parameters in order to achieve the best accuracy on a validation split of 15%. 

In [ ]:
# Package Imports
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

#### 1 hidden layer hyper parameter tuning. 



In [ ]:
one_hidden_layer_tuning = []

#### Optimizer tuning

In [ ]:
X.head()

In [ ]:
# Function to create model, required for KerasClassifier

# Number of neurons
# dropout rate
# activation function

# This is done because tuning more than 3 hyper parameters causes
# the host machine to crash

# The best hyper parameters gotten will be used to tune the learning rate and optimizer next
def dnn_model(dropout_rate=0.0,activation='relu',neurons=1,optimizer='adam',learning_rate=0.001):
    # create model
    model = Sequential()
    model.add(Dense(units=neurons, input_dim=input_layer_size, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=dnn_model, epochs=30, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learning_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
neurons = [1, 5, 10, 15, 20, 25, 30,32,64]
param_grid = dict(activation=activation,dropout_rate=dropout_rate,
                 neurons=neurons,optimizer=optimizer, learning_rate=learning_rate)

# pass parameters to grid search

grid_cv = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=7)
grid_cv_result = grid_cv.fit(X, y, validation_split=0.15)

# summarize results
print("Best: %f using %s" % (grid_cv_result.best_score_, grid_cv_result.best_params_))
means = grid_cv_result.cv_results_['mean_test_score']
stds = grid_cv_result.cv_results_['std_test_score']
params = grid_cv_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#     saving the best parameters
one_hidden_layer_tuning.append(grid_cv_result.best_params_)

In [ ]:
# Function to create model, required for KerasClassifier
# using the best parameters gotten from the previous results
# To get the best optimizer for a one layer network
def dnn_model(optimizer='adam'):
    # create model
    model = Sequential()
    model.add(Dense(units=one_hidden_layer_tuning[0]['neurons'], input_dim=input_layer_size, activation=one_hidden_layer_tuning[0]['activation']))
    model.add(Dropout(one_hidden_layer_tuning[0]['dropout_rate']))
    model.add(Dense(units=1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=dnn_model, epochs=30, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(optimizer=optimizer)

# pass parameters to grid search

grid_cv = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=7)
grid_cv_result = grid_cv.fit(X, y, validation_split=0.15)

# summarize results
print("Best: %f using %s" % (grid_cv_result.best_score_, grid_cv_result.best_params_))
means = grid_cv_result.cv_results_['mean_test_score']
stds = grid_cv_result.cv_results_['std_test_score']
params = grid_cv_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#     saving the best parameters
one_hidden_layer_tuning.append(grid_cv_result.best_params_)

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Function to create model, required for KerasClassifier
# using the best parameters gotten from the previous results
# To get the best learning rate for a one layer network

def dnn_model(learn_rate=0.01):
    # create model
    model = Sequential()
    model.add(Dense(units=one_hidden_layer_tuning[0]['neurons'], input_dim=input_layer_size, activation=one_hidden_layer_tuning[0]['activation']))
    model.add(Dropout(one_hidden_layer_tuning[0]['dropout_rate']))
    model.add(Dense(units=1, activation='sigmoid'))
    # Compile model
    optimizer = RMSprop(learning_rate= learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=dnn_model, epochs=30, batch_size=10, verbose=0)
# define the grid search parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(learn_rate=learn_rate)

# pass parameters to grid search

grid_cv = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=7)
grid_cv_result = grid_cv.fit(X, y, validation_split=0.15)

# summarize results
print("Best: %f using %s" % (grid_cv_result.best_score_, grid_cv_result.best_params_))
means = grid_cv_result.cv_results_['mean_test_score']
stds = grid_cv_result.cv_results_['std_test_score']
params = grid_cv_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#     saving the best parameters
one_hidden_layer_tuning.append(grid_cv_result.best_params_)

In [ ]:
print(one_hidden_layer_tuning)

In [ ]:
two_hidden_layer_tuning =[]

In [ ]:
# Function to create model, required for KerasClassifier
# using the best parameters gotten from the previous results
# To get the best learning rate for a two layer network

def dnn_model():
    # create model
    model = Sequential()
    model.add(Dense(units=one_hidden_layer_tuning[0]['neurons'], input_dim=input_layer_size, activation=one_hidden_layer_tuning[0]['activation']))
    model.add(Dropout(one_hidden_layer_tuning[0]['dropout_rate']))
    model.add(Dense(units=one_hidden_layer_tuning[0]['neurons'], input_dim=input_layer_size, activation=one_hidden_layer_tuning[0]['activation']))
    model.add(Dropout(one_hidden_layer_tuning[0]['dropout_rate']))
    model.add(Dense(units=1, activation='sigmoid'))
    # Compile model
    optimizer = RMSprop(learning_rate= learn_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=dnn_model, epochs=30, batch_size=10, verbose=0)
# define the grid search parameters
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(learn_rate=learn_rate)

# pass parameters to grid search

grid_cv = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=7)
grid_cv_result = grid_cv.fit(X, y, validation_split=0.15)

# summarize results
print("Best: %f using %s" % (grid_cv_result.best_score_, grid_cv_result.best_params_))
means = grid_cv_result.cv_results_['mean_test_score']
stds = grid_cv_result.cv_results_['std_test_score']
params = grid_cv_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

#     saving the best parameters
one_hidden_layer_tuning.append(grid_cv_result.best_params_)

### Question 3

According to the question, the 1D data has equation y = θ_x + θ_0. Solving for x and equating y to 0 will result in the following equation. x = - θ_0/θ and since λ > 0, the following is possible

[A]: Neither high λ or low λ will bring about the prediction rule for A. From the equation above, if λ was negative, it could affect the equation to bring about the linear prediction rule going in the direction of A, but λ > 0. 

[B]: It could be a normal regression line. A low value of λ( close to 0) could bring about B  by fitting the perfectly to the dataset similar to line B.

[C]: Could be produce through a high value of λ by rigde regression. An increase (= infinity) in the ridge regression penalty (λ) will bring about the slope shifting towards a perfect horizontal line similar to line C.

[D]: Neither high λ or low λ will cause the prediction rule for D since the highest possible value of lambda will only shift the prediction rule to an horizontal line (slope = 0).

### Question 4
From the question, the regularization approach is trying to maximize θ<sub>2<sub>. 

Maximizing θ_2 from equation of any linear regression line will result in x_2 = -θ_1x_1/ θ_2

[L2]: No, it can't result from the regularization of θ_2 because when θ_2 is regularized, the boundary formed depends less on x2's value making it be more vertical but L2 looks more horizontal. So penalizing θ_2 will not result in L2.

[L3]: Yes, regularizing θ_2 could result in L3 boundary as regularizing θ_2 will result in L1
coming downwards which could result if C regularization parameter is large.

[L4]: No, it can not result from the regularization of θ_2 as this is not the best solution that can be constructed while keeping w2 small and so L4 is not the best solution when regularizing θ_2.